In [1]:
# import modules
import requests
import shutil
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import urllib
import pickle
import time
from __future__ import print_function

%matplotlib inline

In [2]:
api_key = '5eef6423b6f7cea2f5b6cae0838e38f9'

## Get Dictionary Mapping Genre_id to Genre_name

In [3]:
# API Key (Harry's)
payload = {
            'api_key': '5eef6423b6f7cea2f5b6cae0838e38f9'
         }

r = requests.get("https://api.themoviedb.org/3/genre/movie/list", params = payload).json()

In [4]:
genres = {}
for result in r['genres']:
    genres[result["id"]] = result["name"]

In [5]:
# get rid of u's in front of genre id's
for key in genres.keys():
    genres[key] = genres[key].encode('utf-8').replace(" ", "_")

In [6]:
genres

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science_Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV_Movie'}

In [7]:
# pickle genres
with open('genre_dict.pickle', 'wb') as handle:
    pickle.dump(genres, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Get Data from TMDb

In [9]:
data = {}

In [10]:
from tqdm import *

for genre in genres.keys():
    # get data for genre
    genre_data = []
    for i in tqdm(range(1, 55)):
        url = "https://api.themoviedb.org/3/discover/movie"
        payload = {
                    'api_key': api_key,
                    'with_genres': str(genre),
                    'page': i
                 }
        r = requests.get(url, params = payload).json()
        genre_data += r["results"]
        time.sleep(1)
    # store the genre data
    name_genre = genres[genre]
    data[name_genre] = genre_data

100%|██████████| 54/54 [01:01<00:00,  1.12s/it]


In [18]:
missing_genres = [10770, 10749, 9648, 80]

In [17]:
genres

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science_Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV_Movie'}

## Pickle TMDb Data

In [22]:
with open('data_by_genre.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# test pickling
with open('data_by_genre.pickle', 'rb') as handle:
    data_pickle = pickle.load(handle)

## Create Directories to Store Posters

In [14]:
from os import makedirs, listdir
# for genre_name in genres.values():
#     makedirs("posters/" + genre_name)

## Get 1000 Posters for each Genre based on TMDb Data

In [16]:
for genre, name_genre in genres.items():
# iterate through the genre data and store posters in appropriate directory
    print(name_genre)
    makedirs("posters/" + name_genre)
    counter = 0
    for movie in data[name_genre]:
        if len(listdir("posters/" + name_genre)) == 1000:
            break
        elif movie['poster_path']:
            url = 'http://image.tmdb.org/t/p/w500/' + movie['poster_path']
            
            # build filename
            title = movie["title"].encode('utf-8').replace("/", " ")
            genre_ids = "_".join(str(id) for id in movie["genre_ids"])
            movie_id = str(movie["id"])
            path = "posters"+ "/" + name_genre + "/" + title + "-" + movie_id + "_"+ genre_ids + ".jpg"
            
            urllib.urlretrieve(url, path)
            
            # to keep track of progress                   
            counter += 1
            print(counter, end=', ')

War
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [26]:
for genre_data in data.values():
    assert(len(genre_data) >=1000)

## Put Movie Data in DataFrame

In [20]:
df = pd.DataFrame()
df["movie_id"] = pd.Series()
df["movie_title"] = pd.Series()
df["overview"] = pd.Series()
df["genre_id"] = pd.Series()

In [27]:
index = 0
for genre_data in data.values():
    for movie in genre_data:
        df.loc[index] = (int(movie["id"]), movie["title"].encode("utf-8"),
                          movie["overview"].encode("utf-8"),
                          "-".join(str(id) for id in movie["genre_ids"]))
        index += 1

In [28]:
df["movie_id"] = df["movie_id"].astype(int)
df.index = df['movie_id']
df.head()

,movie_id,movie_title,overview,genre_id
movie_id,,,,
329865,329865,Arrival,Taking place after alien crafts land around th...,53-18-878-9648
210577,210577,Gone Girl,With his wife's disappearance having become th...,9648-53-18
27205,27205,Inception,"Cobb, a skilled thief who commits corporate es...",28-53-878-9648-12
158852,158852,Tomorrowland,"Bound by a shared destiny, a bright, optimisti...",28-10751-878-12-9648
198663,198663,The Maze Runner,"Set in a post-apocalyptic world, young Thomas ...",28-9648-878-53


## Split genre ID's and apply one-hot encoding



In [29]:
split_genres = [map(int, genre_strings.split("-")) for genre_strings in df["genre_id"]]

In [30]:
df["genre_id"] = split_genres

In [31]:
df.head()

,movie_id,movie_title,overview,genre_id
movie_id,,,,
329865,329865,Arrival,Taking place after alien crafts land around th...,"[53, 18, 878, 9648]"
210577,210577,Gone Girl,With his wife's disappearance having become th...,"[9648, 53, 18]"
27205,27205,Inception,"Cobb, a skilled thief who commits corporate es...","[28, 53, 878, 9648, 12]"
158852,158852,Tomorrowland,"Bound by a shared destiny, a bright, optimisti...","[28, 10751, 878, 12, 9648]"
198663,198663,The Maze Runner,"Set in a post-apocalyptic world, young Thomas ...","[28, 9648, 878, 53]"


In [32]:
df["genre_id"].iloc[0]

[53, 18, 878, 9648]

In [33]:
# one-hot-encoding by genre
for genre_id, genre in genres.items():
    df[genre] = [int(genre_id in genre_list) for genre_list in df['genre_id']]

In [34]:
df.to_csv("full_raw_movie_overviews.csv", index=False)

In [35]:
for genre_movies in data.values():
    for movie in genre_movies:
        if not movie["poster_path"]:
            print(movie["title"])

The Lost Symbol
Secret in Suburbia
Thittam Pottu Thirudura Koottam
On Wheels
The Expendables 4
Journey 3: From the Earth to the Moon
Joy in Moscow
Across the Andes by Frog
Busty Cops: Protect and Serve!
Killer Instinct
Le paria
Rat in the Skull
The Care of Time
Upon This Rock
Dark Secrets
Eyewitness
L'éducation sentimentale
Sticks and Bones
Two Kinds Of Love
Rebellkirurgen
The Facts of Life Goes to Paris
June Brown at 90: A Walford Legend
Chiemgauer Volkstheater- Kein Auskommen mit dem Einkommen
The One That Got Away
Tagget
My Husband Is Missing
Zärtliche Erpresserin
Et godt liv
L'homme qui revient de loin
Acts of Love and Other Comedies
The 44th President: In His Own Words
Fast & Furious 10
November Criminals
Secret in Suburbia
Brindavanam
Journey 3: From the Earth to the Moon
Salam
The Swan Princess 7
Beavers: Invasion of the End of the World
Médée
Laali Ki Shaadi Mein Laaddoo Deewana
Blue Mountains
Saturday Church
Depeche Mode - The 6 Music Festival 2017
Abby Singer/Songwriter
Larri